In [1]:
# Core
import numpy as np
import pandas as pd
import os
import subprocess
from __future__ import division
import pickle

def install_package(name):
    sudoPassword = ''
    command = 'pip install ' + name
    p = os.system('echo %s|sudo -S %s' % (sudoPassword, command))

# Stats
from statsmodels.regression import linear_model
import statsmodels.api as sm
import scipy.stats as stats


# ML
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import Lasso
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import AdaBoostClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.utils import shuffle

# OR
from skcriteria import Data, MIN, MAX
from skcriteria.madm import closeness, simple

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns
from ggplot import *

import matplotlib
font = {'family' : 'normal',
  'weight' : 'bold',
  'size' : 16}
matplotlib.rc('font', **font)

/Library/Python/2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
"""
# Read in main data
main_df = pd.read_csv("../../data/Series3_6.15.17_padel.csv")
placebo_8 = pd.read_csv("../../data/Selleck_filtered_padel_corrected.csv")
main_df_imp = pd.read_csv("../../data/Imputed_Series3_6.15.17_padel.csv")
placebo_8_imp = pd.read_csv("../../data/Imputed_Selleck_filtered_padel_corrected.csv")

# Get valid columns
valid_columns = main_df_imp.columns.tolist()

# Gather our dataframes for collective manipulation
frames = [main_df[valid_columns],   placebo_8[valid_columns]]
frames_imp = [main_df_imp, placebo_8_imp]

# Set the index, drop the Name column.
frames[0].set_index('Name', inplace=True)
frames[1].set_index('Name', inplace=True)
frames_imp[0].set_index('Name', inplace=True)
frames_imp[1].set_index('Name', inplace=True)




print(frames[0].shape)
print(frames[1].shape)
print(frames_imp[0].shape)
print(frames_imp[1].shape)

def categorize40(row):
    returnBin=1
    if row > 40.0:
        returnBin=0
    return returnBin

def categorize10(row):
    returnBin=1
    if row >= 10.0:
        returnBin=0
    return returnBin

frames[0].insert(1, 'bin40', frames[0]["IC50"].transform(categorize40).astype('category') )
frames[1].insert(1, 'bin40', frames[1]["IC50"].transform(categorize40).astype('category') )
frames_imp[0].insert(1, 'bin40', frames_imp[0]["IC50"].transform(categorize40).astype('category') )
frames_imp[1].insert(1, 'bin40', frames_imp[1]["IC50"].transform(categorize40).astype('category') )

frames[0].insert(2, 'bin10', frames[0]["IC50"].transform(categorize10).astype('category') )
frames[1].insert(2, 'bin10', frames[1]["IC50"].transform(categorize10).astype('category') )
frames_imp[0].insert(2, 'bin10', frames_imp[0]["IC50"].transform(categorize10).astype('category') )
frames_imp[1].insert(2, 'bin10', frames_imp[1]["IC50"].transform(categorize10).astype('category') )
"""

(47, 612)
(2796, 612)
(47, 612)
(2796, 612)


In [5]:
#pickle.dump((frames[0],frames[1]), open("../pickled/final_data.pickle","w"))
Xtrain, Xtest = pickle.load(open("../pickled/final_data.pickle","rw"))
#pickle.dump((frames_imp[0],frames_imp[1]), open("../pickled/final_data_imputed.pickle","w"))
Xtrain_imp, Xtest_imp = pickle.load(open("../pickled/final_data_imputed.pickle","rw"))

print (Xtrain.shape)
print (Xtest.shape)
print (Xtrain_imp.shape)
print (Xtest_imp.shape)



(47, 614)
(2796, 614)
(47, 614)
(2796, 614)


In [6]:
#Xtrain = Xtrain[["IC50","bin10","bin40"]+Xtrain.columns.tolist()[3:]]
Xtest = Xtest[["IC50","bin10","bin40"]+Xtest.columns.tolist()[3:]]
#Xtrain_imp = Xtrain_imp[["IC50","bin10","bin40"]+Xtrain_imp.columns.tolist()[3:]]
Xtest_imp = Xtest_imp[["IC50","bin10","bin40"]+Xtest_imp.columns.tolist()[3:]]

Xtrain.head()

,IC50,bin40,bin10,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,...,AMW,WTPT-1,WTPT-2,WTPT-3,WTPT-4,WTPT-5,WPATH,WPOL,XLogP,Zagreb
Name,,,,,,,,,,,,,,,,,,,,,
OSM-S-106,0.036,1,1,-1.0958,1.200778,23.1572,39.591930,15.0,17.0,30.0,...,10.200817,40.713610,2.035681,22.242022,4.765434,11.047825,799.0,31.0,0.727,110.0
OSM-S-107,10.000,1,0,1.1547,1.333332,38.0230,49.112481,15.0,16.0,40.0,...,8.202895,48.001779,2.087034,18.848454,0.000000,13.231952,1321.0,34.0,3.185,124.0
OSM-S-118,50.000,0,0,0.5906,0.348808,7.7935,31.554344,15.0,17.0,23.0,...,9.219166,31.354825,2.090322,9.306953,0.000000,6.214417,343.0,21.0,4.191,80.0
OSM-S-119,50.000,0,0,2.2246,4.948845,25.3114,36.320758,15.0,17.0,23.0,...,15.993993,35.091951,2.064232,14.383663,0.000000,6.220694,480.0,27.0,5.059,92.0
OSM-S-120,50.000,0,0,1.3358,1.784362,16.7123,33.937551,15.0,17.0,23.0,...,12.606580,33.230029,2.076877,11.846725,0.000000,6.220707,411.0,23.0,4.625,86.0


In [70]:
# FEATURE SPACES
rf_pipe              = Pipeline([('rf',RandomForestClassifier())])
rf_param_grid       = {'rf__n_estimators':[10,100],
                       'rf__bootstrap':[True, False],
                       'rf__criterion':['gini','entropy']}

rf_grid             = GridSearchCV(rf_pipe, param_grid=rf_param_grid, n_jobs=4, cv=3, verbose=True)

rf_fit              = rf_grid.fit(Xtrain.iloc[:,3:],Xtrain.bin40).best_estimator_.named_steps['rf']
rf_features         = rf_fit.feature_importances_
rf_features         = Xtrain.columns[rf_features > 0].tolist()
rf_features_size    = len(rf_features)
rf_features[:10]

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=4)]: Done  24 out of  24 | elapsed:    2.0s finished


['nX',
 'ATS1v',
 'ATS5v',
 'ATS1p',
 'ATS2p',
 'ATS4s',
 'AATS3m',
 'AATS7m',
 'AATS1i',
 'AATS4s']

In [13]:
# REGRESSION ON IMPUTED DATA
# MODEL 1 RFR
"""
rf_pipe          = Pipeline([('scaler', StandardScaler()),('rf',RandomForestRegressor())])
rf_param_grid    = {'rf__n_estimators':[10,100,1000],
                   'rf__bootstrap':[True, False]}

rf_grid             = GridSearchCV(rf_pipe, param_grid=rf_param_grid, cv=3,n_jobs=4,verbose=True, scoring="r2")

rf_fit=rf_grid.fit(Xtrain_imp.iloc[:,3:],Xtrain_imp.IC50)
best_rf = rf_fit.best_estimator_.named_steps["rf"]
best_rf_1 = np.mean(cross_val_score(best_rf, Xtest_imp.iloc[:,3:], Xtest_imp.IC50, cv=3, scoring='r2'))
"""
print("RFR (IMPUTED):" , best_rf_1) #1.0

# MODEL 2 SVR
from sklearn.svm import SVR
"""
svr_pipe          = Pipeline([('scaler', StandardScaler()),('svr',SVR())])
svr_param_grid    = {'svr__kernel':['rbf', 'linear', 'poly'],
                     'svr__C':[10, 1.0, 0.1, 0.001],
                     'svr__epsilon':[10,1,0.1,0.01, 0.001]}

svr_grid             = GridSearchCV(svr_pipe, param_grid=svr_param_grid, cv=3,n_jobs=4,verbose=True, scoring="r2")

svr_fit=svr_grid.fit(Xtrain_imp.iloc[:,3:],Xtrain_imp.IC50)
best_svr = svr_fit.best_estimator_.named_steps["svr"]
best_svr_1 = np.mean(cross_val_score(best_svr, Xtest_imp.iloc[:,3:], Xtest_imp.IC50, cv=3, scoring='r2'))
"""
print("SVR: (IMPUTED)" , best_svr_1) #1.0
# CLASSIFICATION ON IMPUTED DATA
# MODEL 1 RFC
# MODEL 2 SVC

('RFR:', 1.0)
Fitting 3 folds for each of 60 candidates, totalling 180 fits
('SVR:', 1.0)


[Parallel(n_jobs=4)]: Done 180 out of 180 | elapsed:    1.0s finished


SVR(C=0.1, cache_size=200, coef0=0.0, degree=3, epsilon=10, gamma='auto',
  kernel='poly', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [26]:
# REGRESSION ON NON-IMPUTED DATA (REPLACED BY 0)
# MODEL 1 RFR
"""
rf_pipe          = Pipeline([('scaler', StandardScaler()),('rf',RandomForestRegressor())])
rf_param_grid    = {'rf__n_estimators':[10,100,1000],
                   'rf__bootstrap':[True, False]}

rf_grid             = GridSearchCV(rf_pipe, param_grid=rf_param_grid, cv=3,n_jobs=4,verbose=True, scoring="r2")

rf_fit=rf_grid.fit(Xtrain.iloc[:,3:],Xtrain.IC50)
best_rf = rf_fit.best_estimator_.named_steps["rf"]
best_rf_2 = np.mean(cross_val_score(best_rf, Xtest.iloc[:,3:], Xtest.IC50, cv=3, scoring='r2'))
"""
print("RFR (NON IMPUTED):" , best_rf_2) #1.0

# MODEL 2 SVR
from sklearn.svm import SVR
"""
svr_pipe          = Pipeline([('scaler', StandardScaler()),('svr',SVR())])
svr_param_grid    = {'svr__kernel':['rbf', 'linear', 'poly'],
                     'svr__C':[10, 1.0, 0.1, 0.001],
                     'svr__epsilon':[10,1,0.1,0.01, 0.001]}

svr_grid             = GridSearchCV(svr_pipe, param_grid=svr_param_grid, cv=3,n_jobs=4,verbose=True, scoring="r2")

svr_fit=svr_grid.fit(Xtrain.iloc[:,3:],Xtrain.IC50)
best_svr = svr_fit.best_estimator_.named_steps["svr"]
best_svr_2 = np.mean(cross_val_score(best_svr, Xtest.iloc[:,3:], Xtest.IC50, cv=3, scoring='r2'))
"""
print("SVR (NON IMPUTED):", best_svr_1)
# CLASSIFICATION ON NON-IMPUTED DATA (REPLACED BY 0)
# MODEL 1 RFC
rf_pipe              = Pipeline([('scaler', StandardScaler()),('rf',RandomForestClassifier())])
rf_param_grid       = {'rf__n_estimators':[10,100],
                       'rf__bootstrap':[True, False],
                       'rf__criterion':['gini','entropy']}

rf_grid             = GridSearchCV(rf_pipe, param_grid=rf_param_grid, n_jobs=4, cv=3, verbose=True)

best_rfc            = rf_grid.fit(Xtrain.iloc[:,3:],Xtrain.bin40).best_estimator_.named_steps['rf']
best_rfc_2 = np.mean(cross_val_score(best_rfc, Xtest.iloc[:,3:], Xtest.bin40, cv=3, scoring='roc_auc'))

print("RFC (NON IMPUTED):", best_rfc_2)

# MODEL 2 SVC
svc_pipe              = Pipeline([('scaler', StandardScaler()),('svc',SVC())])
svc_param_grid        = {'svc__kernel':['rbf', 'linear', 'poly'],
                         'svc__C':[10, 1.0, 0.1, 0.01]}

svc_grid             = GridSearchCV(svc_pipe, param_grid=svc_param_grid, n_jobs=4, cv=3, verbose=True)

best_svc              = svc_grid.fit(Xtrain.iloc[:,3:],Xtrain.bin40).best_estimator_.named_steps['svc']
best_svc_2 = np.mean(cross_val_score(best_svc, Xtest.iloc[:,3:], Xtest.bin40, cv=3, scoring='roc_auc'))
print("SVC (NON IMPUTED):", best_svc_2)



('RFR (NON IMPUTED):', 1.0)
('SVR (NON IMPUTED):', 1.0)
Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=4)]: Done  36 out of  36 | elapsed:    0.3s finished


ValueError: The number of classes has to be greater than one; got 1

In [24]:
Xtrain.iloc[:,3:].head()

,bin10,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,nHeavyAtom,nH,...,AMW,WTPT-1,WTPT-2,WTPT-3,WTPT-4,WTPT-5,WPATH,WPOL,XLogP,Zagreb
Name,,,,,,,,,,,,,,,,,,,,,
OSM-S-106,1,-1.0958,1.200778,23.1572,39.591930,15.0,17.0,30.0,20.0,10.0,...,10.200817,40.713610,2.035681,22.242022,4.765434,11.047825,799.0,31.0,0.727,110.0
OSM-S-107,0,1.1547,1.333332,38.0230,49.112481,15.0,16.0,40.0,23.0,17.0,...,8.202895,48.001779,2.087034,18.848454,0.000000,13.231952,1321.0,34.0,3.185,124.0
OSM-S-118,0,0.5906,0.348808,7.7935,31.554344,15.0,17.0,23.0,15.0,8.0,...,9.219166,31.354825,2.090322,9.306953,0.000000,6.214417,343.0,21.0,4.191,80.0
OSM-S-119,0,2.2246,4.948845,25.3114,36.320758,15.0,17.0,23.0,17.0,6.0,...,15.993993,35.091951,2.064232,14.383663,0.000000,6.220694,480.0,27.0,5.059,92.0
OSM-S-120,0,1.3358,1.784362,16.7123,33.937551,15.0,17.0,23.0,16.0,7.0,...,12.606580,33.230029,2.076877,11.846725,0.000000,6.220707,411.0,23.0,4.625,86.0


In [22]:
Xtest.shape

(2796, 614)

1.0

In [12]:
X = Xtest.iloc[:,3:].as_matrix()
for i in range(len(X)):
    for j in range(len(X[i])):
        if X[i,j] == "#NAME?" or X[i,j] == np.nan or X[i,j] == np.inf:
            X[i,j] = 0

Xtest.iloc[:,3:] = X

In [9]:
Xtest

,IC50,bin10,bin40,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,...,AMW,WTPT-1,WTPT-2,WTPT-3,WTPT-4,WTPT-5,WPATH,WPOL,XLogP,Zagreb
Name,,,,,,,,,,,,,,,,,,,,,
ABT-888 (Veliparib),250.0,0,0,-1.3418,1.800427,34.1526,38.750688,9,10,34,...,7.180365,36.743637,2.041313,14.172273,2.430358e+00,11.741915,5.650000e+02,28,0.885,100.0
Axitinib,250.0,0,0,0.1055,0.011130,34.5503,58.824274,21,22,46,...,8.393916,57.883110,2.067254,17.697744,2.494923e+00,12.032997,2.325000e+03,42,5.771,146.0
Saracatinib (AZD0530),250.0,0,0,-0.8460,0.715716,79.6564,80.547376,16,17,70,...,7.731560,79.497342,2.092035,34.177216,1.548092e+01,16.119589,4.855000e+03,61,2.174,206.0
AZD6244 (Selumetinib),250.0,0,0,0.5819,0.338608,48.0315,52.514895,15,16,42,...,10.857143,54.475907,2.017626,28.064240,7.913593e+00,12.480696,1.808000e+03,44,3.017,140.0
Nintedanib (BIBF 1120),250.0,0,0,-1.7778,3.160573,79.1978,85.272169,18,18,73,...,7.387031,81.997445,2.049936,26.514870,1.034128e+01,16.173592,5.935000e+03,66,4.975,212.0
Bosutinib (SKI-606),250.0,0,0,0.7555,0.570780,79.6580,77.362997,16,17,65,...,8.140996,73.414653,2.039296,29.250489,8.722063e+00,15.440191,4.488000e+03,60,2.571,186.0
Cediranib (AZD2171),250.0,0,0,-0.6656,0.443023,50.9406,69.366411,19,21,60,...,7.503445,68.465460,2.074711,24.388201,9.115172e+00,12.677339,3.562000e+03,52,4.002,178.0
CI-1040 (PD184352),250.0,0,0,2.1312,4.542013,50.0860,51.703102,12,12,39,...,12.255786,50.005795,2.000232,21.780568,5.515989e+00,6.119944,1.553000e+03,38,5.487,130.0
Dasatinib (BMS-354825),250.0,0,0,0.6087,0.370516,72.1852,70.440618,17,17,59,...,8.256877,67.333454,2.040408,33.006303,5.006277e+00,22.344994,3.723000e+03,50,2.220,172.0
